In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train[train['target']==1][['duration', 'campaign']].describe()

,duration,campaign
count,2787.000000,2787.000000
mean,551.896304,2.049157
std,404.420029,1.617637
min,37.000000,1.000000
25%,255.000000,1.000000
50%,442.000000,2.000000
75%,739.500000,2.000000
max,4199.000000,17.000000


In [4]:
test = pd.read_csv('test.csv')

In [5]:
test.head()

,_id,age,job,marital,education,default,housing,loan,contact,month,...,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,66810d8e6bf2b41c880a7bc6c8a1e295,57,technician,married,high.school,no,no,yes,cellular,may,...,371,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1
1,ccac3879652b08cb8b44c1920fd93afa,55,unknown,married,unknown,unknown,yes,no,telephone,may,...,285,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0
2,fcccab4d7a76f70647f015f2c84c2af8,33,blue-collar,married,basic.9y,no,no,no,cellular,may,...,52,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1
3,ed8399278c30678dab739045fa12b440,36,admin.,married,high.school,no,no,no,telephone,jun,...,355,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1
4,1d4d62ac5cabcb48bac7112813f290cb,27,housemaid,married,high.school,no,yes,no,cellular,jul,...,189,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1


In [6]:
y = train['target']

In [7]:
train = train.drop('target', axis=1)

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24712 entries, 0 to 24711
Data columns (total 21 columns):
_id               24712 non-null object
age               24712 non-null int64
job               24712 non-null object
marital           24712 non-null object
education         24712 non-null object
default           24712 non-null object
housing           24712 non-null object
loan              24712 non-null object
contact           24712 non-null object
month             24712 non-null object
day_of_week       24712 non-null object
duration          24712 non-null int64
campaign          24712 non-null int64
pdays             24712 non-null int64
previous          24712 non-null int64
poutcome          24712 non-null object
emp.var.rate      24712 non-null float64
cons.price.idx    24712 non-null float64
cons.conf.idx     24712 non-null float64
euribor3m         24712 non-null float64
nr.employed       24712 non-null float64
dtypes: float64(5), int64(5), object(11)
memory usa

In [9]:
def getDurationClass(d, df):
    durationStd = df.duration.std()
    durationMin = df.duration.min()
    durationMax = df.duration.max()
    return {
#          (100 <= d) : True,
        (100 <= d <= 700) : True,
        (d < 100) | (700 < d): False
#         (d < 100): False

    }[True]

def isBad(df):
    return ((df['pdays']==999) | (df['pdays']==0)) & (df['previous']>0)
    
def preproc_data(df_input):
    df_output = df_input.copy()
    
    df_output = df_output.drop('_id', axis=1)
    
    months = ['jan', 'feb', 'mar', 'apr', 'may', 'jul', 'jun', 'aug', 'sep', 'oct', 'nov','dec']
    df_output['kvartal'] = df_output['month'].apply(lambda m : months.index(m)/3+1)
    
    job = list(set(df_output['job']))
    df_output['job'] = df_output.job.apply(lambda r: job.index(r))

    month = list(set(df_output['month']))
    df_output['month'] = df_output.month.apply(lambda r: month.index(r))   
    
    day_of_week = list(set(df_output['day_of_week']))
    df_output['day_of_week'] = df_output.day_of_week.apply(lambda r: day_of_week.index(r)) 
    df_output['work_day'] = df_output.day_of_week.apply(lambda r: r < 6) 

#     df_output['is_bad'] = df_output[['pdays', 'previous']].apply(lambda x : isBad(x), axis=1)
#     df_output['has_contact'] = df_output['pdays'].apply(lambda d: d != 999)
    df_output['pdays'] = df_output['pdays'].apply(lambda d: 0 if d == 999 else d)
    df_output['duration_class'] = df_output['duration'].apply(lambda d: getDurationClass(d, df_output))
#     df_output['duration_class'] = df_output['duration_class'].map(lambda d: True if d & ((df_output['campaign']==1) | (df_output['campaign']==2)) else False)

    #     df_output['duration_class'] = df_output['duration_class', 'campaign'].map() 
    df_output['duration_interrupted'] = df_output['duration'].apply(lambda d: 0 if d < 30 else d)
#     df_output['age_1'] = df_output['duration'].apply(lambda d: d < 30)
#     df_output['age_2'] = df_output['duration'].apply(lambda d: 30 <= d < 60)
#     df_output['age_3'] = df_output['duration'].apply(lambda d: 60 <= d < 120)
#     df_output['duration_interrupted2'] = df_output['duration'].apply(lambda d: 0 if 30 <= d < 100 else d)
#     df_output = df_output.reset_index()

    df_output = pd.get_dummies(df_output, columns=[
#          'job',
#          'month',
#          'day_of_week',
         'marital',
         'education',
         'default',
         'housing',
         'loan',
         'contact',
         'poutcome'
#           'duration_class'
    ])
    
    return df_output

In [10]:
train_work = preproc_data(train)
test_work = preproc_data(test)

In [11]:
train_work.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24712 entries, 0 to 24711
Data columns (total 43 columns):
age                              24712 non-null int64
job                              24712 non-null int64
month                            24712 non-null int64
day_of_week                      24712 non-null int64
duration                         24712 non-null int64
campaign                         24712 non-null int64
pdays                            24712 non-null int64
previous                         24712 non-null int64
emp.var.rate                     24712 non-null float64
cons.price.idx                   24712 non-null float64
cons.conf.idx                    24712 non-null float64
euribor3m                        24712 non-null float64
nr.employed                      24712 non-null float64
kvartal                          24712 non-null int64
work_day                         24712 non-null bool
duration_class                   24712 non-null bool
duration_interrupte

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(train_work, y, test_size=0.33, random_state=42)

In [14]:
from sklearn.tree import DecisionTreeClassifier

In [15]:
from ipywidgets import Image
from io import StringIO
import pydotplus
from sklearn.tree import export_graphviz

# dot_data = StringIO()
# export_graphviz(dtc, feature_names=train_work.columns, out_file=dot_data, filled=True)
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# Image(value=graph.create_png())

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [17]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

classifiers = {
    'tree': DecisionTreeClassifier(max_depth=5), 
    'logistic': LogisticRegression(),
    'forest': RandomForestClassifier(),
    'kn': KNeighborsClassifier(n_neighbors=10)
}

# splitCount = 10
# cv = StratifiedKFold(n_splits=splitCount, shuffle=True, random_state=123)

# for classifierName, classifier in classifiers.iteritems():
#     roc_auc = 0
#     log_loss_ = 0
    
#     for train_index, test_index in cv.split(train_work, y):
#         X_train_kfold, X_test_kfold = train_work.iloc[train_index], train_work.iloc[test_index]
#         y_train_kfold, y_test_kfold = y.iloc[train_index], y.iloc[test_index]
#         classifier.fit(X_train_kfold, y_train_kfold)
#         classifierPredictProba = classifier.predict_proba(X_test_kfold)
#         roc_auc += roc_auc_score(y_test_kfold, classifierPredictProba[:,1])
#         log_loss_ += log_loss(y_test_kfold, classifierPredictProba[:,1])
#     roc_auc = roc_auc / splitCount 
#     log_loss_ = log_loss_ / splitCount 
#     print(classifierName, roc_auc, log_loss_)

In [18]:
import numpy as np
dtc = DecisionTreeClassifier(max_depth=5)
dtc.fit(X_train, y_train)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

lr = LogisticRegression()
lr.fit(X_train, y_train)

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, train_work.columns[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature duration_interrupted (0.170180)
2. feature duration (0.166588)
3. feature euribor3m (0.089414)
4. feature age (0.066522)
5. feature nr.employed (0.055164)
6. feature job (0.036997)
7. feature cons.conf.idx (0.033326)
8. feature day_of_week (0.032902)
9. feature pdays (0.032163)
10. feature campaign (0.031836)
11. feature emp.var.rate (0.026755)
12. feature cons.price.idx (0.026446)
13. feature month (0.019783)
14. feature poutcome_success (0.019082)
15. feature education_university.degree (0.012137)
16. feature kvartal (0.011976)
17. feature housing_yes (0.011241)
18. feature duration_class (0.011221)
19. feature housing_no (0.010975)
20. feature marital_married (0.010576)
21. feature education_high.school (0.010496)
22. feature previous (0.010452)
23. feature marital_single (0.009803)
24. feature loan_no (0.007741)
25. feature education_professional.course (0.007711)
26. feature contact_cellular (0.007536)
27. feature loan_yes (0.007508)
28. feature poutcom

In [19]:
roc_auc_score(y_test, dtc.predict_proba(X_test)[:,1])

0.9191873201001832

In [20]:
roc_auc_score(y_test, lr.predict_proba(X_test)[:,1])

0.9219085641658257

In [21]:
roc_auc_score(y_test, knn.predict_proba(X_test)[:,1])

0.816631378266233

https://www.kaggle.com/cast42/exploring-features

In [22]:
from sklearn.metrics import log_loss
log_loss(y_test, dtc.predict_proba(X_test)[:,1])

0.2508944027535095

In [23]:
log_loss(y_test, lr.predict_proba(X_test)[:,1])

0.2214509387494303

In [24]:
log_loss(y_test, knn.predict_proba(X_test)[:,1])

1.4806464461019222

In [25]:
train_work_tmp = train_work.copy()
train_work_tmp['dtc'] = dtc.predict_proba(train_work)[:,1]
# train_work_tmp['knn'] = knn.predict_proba(train_work)[:,1]
train_work_tmp['rf'] = rf.predict_proba(train_work)[:,1]
train_work = train_work_tmp

test_work_tmp = test_work.copy()
test_work_tmp['dtc'] = dtc.predict_proba(test_work)[:,1]
# test_work_tmp['knn'] = knn.predict_proba(test_work)[:,1]
test_work_tmp['rf'] = rf.predict_proba(test_work)[:,1]
test_work = test_work_tmp

In [26]:
train_work.head()

,age,job,month,day_of_week,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,...,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,poutcome_failure,poutcome_nonexistent,poutcome_success,dtc,rf
0,54,9,1,3,87,1,0,0,1.4,93.444,...,1,0,0,1,0,0,1,0,0.002507,0.0
1,36,7,3,4,291,1,0,0,1.1,93.994,...,1,0,0,0,1,0,1,0,0.024198,0.0
2,53,11,1,4,182,1,0,0,1.4,93.444,...,1,0,0,1,0,0,1,0,0.002507,0.0
3,34,11,3,2,180,2,0,0,1.1,93.994,...,0,0,1,0,1,0,1,0,0.002507,0.0
4,29,7,1,1,6,1,0,0,-1.7,94.027,...,1,0,0,0,1,0,1,0,0.000000,0.0


In [27]:
X_train, X_test, y_train, y_test = train_test_split(train_work, y, test_size=0.33, random_state=42)
lr.fit(X_train, y_train)
roc_auc_score(y_test, lr.predict_proba(X_test)[:,1])

0.9428920040372323

In [28]:
log_loss(y_test, lr.predict_proba(X_test)[:,1])

0.2020172362160242

In [29]:
result_predict = pd.DataFrame(data={'_id': test['_id'], 'target': lr.predict_proba(test_work)[:,1]})

In [30]:
result_predict.to_csv('dsaponenko_03_predict.csv', index=False)